In [ ]:
## importing stuff
import os
import pickle
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
# %aimport islets

from IPython.display import display
import pandas as pd

from islets.Recording import Recording, saveMovie
from islets.Regions import Regions
from islets.numeric import power_spectrum, rebin
from islets.utils import show_movie, saveRois
from islets.Regions import getPeak2BoundaryDF, getGraph_of_ROIs_to_Merge, mergeBasedOnGraph

from caiman import movie as cmovie

from matplotlib.colors import LogNorm

In [ ]:
# from copy import deepcopy
fracSaturTh = .05
# movieFreq = 2
cmap = plt.cm.Greys
cmap.set_bad("lime")
pd.set_option('display.max_rows', 100)

In [ ]:
recFile = "/data/Sandra/2019/2019_10_10/Experiment35a.lif"
rec = Recording(recFile)

In [ ]:
rec.metadata

In [ ]:
ser = rec.metadata.Name.iloc[6]

In [ ]:
pickleFilename = f"{recFile}_analysis/{ser}/"+[f for f in os.listdir(f"{recFile}_analysis/{ser}") if "pkl" in f][0]

In [ ]:
pickleFilename

In [ ]:
with open(pickleFilename, "rb") as f:
    regions = pickle.load(f)

In [ ]:
regions.update()

In [ ]:
regions.Freq

In [ ]:
regions.infer_gain(plot=True)

In [ ]:
ts = 5
regions.fast_filter_traces(ts,z_sp=5)

In [ ]:
regions.calc_peaks(ts,z_th=5)

In [ ]:
# regions.examine()

In [ ]:
regions.show_scatter_peaks(ts)

In [ ]:
raster, = regions.peaks2raster(ts,npoints=len(regions.showTime["%g"%ts]), z_th=5)

In [ ]:
raster.shape

In [ ]:
plt.imshow(raster)

In [ ]:
f"/home/jupyter-mark/{rec.Experiment.split('.')[0]}_{ser}_{1./np.diff(regions.showTime['%g'%ts]).mean():.3g}Hz.txt"

In [ ]:
np.savetxt(
    f"/data/useful_notebooks/{rec.Experiment.split('.')[0]}_{ser}_{1./np.diff(regions.showTime['%g'%ts]).mean():.3g}Hz.txt",
    raster,
    fmt="%i"
          )
# ("")

In [ ]:
# rec.import_series(ser, isLineScan=False, onlyMeta=False)

In [ ]:
metadata = rec.Series[ser]['metadata']

In [ ]:
movie = cmovie(rec.Series[ser]['data'],fr = metadata.Frequency)

In [ ]:
movie.shape

In [ ]:
movie.fr

In [ ]:
meanImage = movie.mean(0)

In [ ]:
plt.imshow(meanImage, norm=LogNorm())

In [ ]:
Nsatur = (movie==movie.max()).sum(0)

In [ ]:
plt.imshow(Nsatur.astype(float),
           norm=LogNorm(),
           aspect="auto"
          )
plt.colorbar()

In [ ]:
plt.rcParams["animation.embed_limit"] = 200

In [ ]:
plt.hist(movie[::100,::10,::10].flatten(),100);

In [ ]:
len(movie)

In [ ]:
pd.DataFrame(columns=["compound","concentration","begin","end"]).to_csv("~/tmp.csv",)

In [ ]:
movie[:100].fr

#### Playing with `high_pass_filter_space`

In [ ]:
# r0 = Regions(cv2.GaussianBlur(exim,(11,11),-1)[:,:50], gSig_filt=None)
r0 = Regions(exim, gSig_filt=None, img_th=.02)
r0.Freq=1

In [ ]:
plt.hist(processed.flat,100);

In [ ]:
fs = 10
processed = high_pass_filter_space(exim,(fs,fs))
# processed = np.maximum(0,processed)
r1 = Regions(processed, gSig_filt=(None), img_th=0-.02)

fig,axs = plt.subplots(1,2,figsize=(14,3))
for ax,r in zip(axs,[r0,r1]):
    im = ax.imshow(exim)
    r.plotEdges(ax=ax, image=False,color='w',lw=.5)
    r.plotPeaks(ax=ax, image=False,color='w',)
    plt.colorbar(im, ax=ax, orientation="horizontal")

In [ ]:
plt.plot(r0.image.flat[::100], r1.image.flat[::100],".")

In [ ]:
# saveMovie(cmovie(rebin(movie[:1000],(2,2),axis=(1,2)), fr=movie.fr),"embed")

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
th = get_sep_th(meanImage.flatten(),log=True,plot=True)

In [ ]:
import cv2

In [ ]:
tmp = cv2.GaussianBlur(meanImage,(5,5),-1,-1)[200:250]

In [ ]:
th = get_sep_th(tmp.flatten()/tmp.max(), plot=True, log=True)

In [ ]:
rr = Regions({"mean":tmp},
             gSig_filt=(None),
             img_th=th,
             processes=1,
             mode="mean",
             diag=True)

In [ ]:
ax = plt.subplot(111)
ix = rr.df.index[:]
rr.plotEdges(ax=ax, ix=ix)
rr.plotPeaks(ax=ax,ix=ix)

In [ ]:
tmp1 = meanImage[200:250]

In [ ]:
th = get_sep_th(tmp1.flatten()/tmp1.max(), plot=True, log=True)

In [ ]:
from caiman.motion_correction import high_pass_filter_space

In [ ]:
hpf = high_pass_filter_space(meanImage,(10,10))

In [ ]:
plt.imshow(hpf)

In [ ]:
cv2.filter2D??

In [ ]:
cv2.filter2D(meanImage,-1,)

In [ ]:
high_pass_filter_space??

In [ ]:
rr1 = Regions({"mean":meanImage},
             gSig_filt=(10,),
             img_th=th,
#              full=False,
#              processes=1,
             mode="mean",
             diag=True)

In [ ]:
ax = plt.subplot(111)
ix = rr1.df.index[:]
rr1.plotEdges(ax=ax, ix=ix)
rr1.plotPeaks(ax=ax,ix=ix)

In [ ]:
tmp2 = cv2.GaussianBlur(meanImage,(5,5),-1,-1)[200:250]-cv2.GaussianBlur(meanImage,(11,11),-1,-1)[200:250]

In [ ]:
th = get_sep_th(tmp2.flatten()/tmp2.max(), plot=True, log=False)

In [ ]:
rr2 = Regions({"mean":tmp2},
             gSig_filt=(None),
             img_th=0,
#              processes=1,
             mode="mean",
             diag=True)

In [ ]:
ax = plt.subplot(111)
ix = rr2.df.index[:]
rr2.plotEdges(ax=ax, ix=ix)
rr2.plotPeaks(ax=ax, ix=ix)

In [ ]:
plt.imshow(rr1.image)

In [ ]:
plt.hist(rr1.image.flatten(),100);

In [ ]:
import cv2

In [ ]:
plt.imshow(meanImage-cv2.GaussianBlur(meanImage,(3,3),-1,-1))

### Check for wiggles

In [ ]:
from islets.numeric import get_sep_th

In [ ]:
from cv2 import GaussianBlur

In [ ]:
im_th = get_sep_th(GaussianBlur(meanImage,(3,3),sigmaX=-1).flatten(), log=True, plot=True)
# plt.axvline(np.log10(.1))

In [ ]:
regions = Regions(meanImage,diag=False,img_th=im_th)

In [ ]:
th = get_sep_th(regions.df.peakValue.values, log=True, plot=True)

In [ ]:
x = np.vstack([movie[:,point[0],point[1]] for point in regions.df.peak])
# x = np.vstack(regions.df.trace)
f,p = power_spectrum(x, fr=metadata.Frequency)
f,p = f[f>0.5],np.abs(p[f>0.5])

In [ ]:
n = 10
plt.plot(rebin(f,n),rebin(p,n))

In [ ]:
# f,p = power_spectrum(np.vstack(data[:,0]),fr=metadata.Frequency,mean=False)

In [ ]:
# plt.boxplot(p[:,2:100],positions=f[2:100], showfliers=False);
# plt.plot(f[2:],p.mean(0)[2:],lw=.6)
# plt.xscale("symlog")

In [ ]:
# plt.plot(*sum([(f,pp) for pp in p[:10]],tuple()),lw=.3)

### Continue

In [ ]:
5/metadata.pxSize

In [ ]:
plt.hist(np.log10(meanImage.flat),100,);

In [ ]:
movie[(slice(None),)+np.where(Nsatur>fracSaturTh*len(movie))] = movie.max()

In [ ]:
regions = Regions(movie, diag=True, gSig_filt=(4), img_th=im_th, )

In [ ]:
# plt.hexbin(regions.statImages["mean"].flatten(),regions.statImages["diff_std"].flatten(), cmap="hot", mincnt=1)

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(10,5))
for k,ax in zip(["mean","diff_std"],axs):
    ax.imshow(regions.statImages[k], norm=LogNorm())
    regions.plotEdges(ax=ax,image=False)
    regions.plotPeaks(ax=ax,image=False)

In [ ]:
show_movie(movie[:], additionalPlot=lambda xi: regions.plotEdges(ax=xi,lw=.7, image=False))

In [ ]:
regions.update(movie)

In [ ]:
regions.detrend_traces()

In [ ]:
regions.infer_gain(plot=True)

In [ ]:
regions.gain

In [ ]:
ts = 4
regions.fast_filter_traces(ts)

In [ ]:
x = np.vstack(regions.df["zScore_%g"%ts])
# x = rebin(x,100,1)
plt.imshow(x>3, aspect="auto")

In [ ]:
regions.df.columns

In [ ]:
regions.fast_filter_traces(10)